# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 3

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task3`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [ ]:
!az login

## Step 1. Write the code

In [1]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

Preparing Q# environment...


In [2]:
teamname="kodex_38"  # Update this field with your team name
task=["task3"]
slack_id="Jeevesh Krishna A"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [3]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task3_DumpMachineWrapper : qsharp.QSharpCallable = None
Task3_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 3 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [18]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

// Task 3. 
// (input will contain 6 qubits)
operation original(input : Qubit[], target : Qubit) : Unit is Adj {
    for i in [7, 11, 13, 14, 19, 21, 22, 25, 26, 28, 35, 37, 38, 41, 42, 44, 49, 50, 52, 56] {
        ControlledOnInt(i, X)(input, target);
    }
}

operation CarryN(carryIn: Qubit, summand1: Qubit, summand2: Qubit, carryOut: Qubit) : Unit is Adj + Ctl {
    CCNOT (summand1, summand2, carryOut);
    CNOT (summand1, summand2);
    CCNOT (carryIn, summand2, carryOut);
    CNOT(carryIn, summand2);
}

operation Task3(input : Qubit[], target : Qubit) : Unit is Adj {
   use sumWires = Qubit[4];
    use zero = Qubit[1];
    within{
    //Ebene 1
    CarryN(input[0], input[1], input[2], sumWires[0]);
    CarryN(input[3], input[4], input[5], sumWires[1]);
    //Ebene 2
    CarryN(zero[0], input[2], input[5], sumWires[2]);
    CarryN(sumWires[2], sumWires[0], sumWires[1], sumWires[3]);
    //Outputs: input[5], sumWires[1], sumWires[3]
    // == 3 --> input[5] == 1 && sumWires[1] == 1 && sumWires[3] == 0
    X(sumWires[3]);
    } apply{
        Controlled X([input[5], sumWires[1], sumWires[3]], target);
    }
}

operation QubitArrayWrapperOperation (op : ((Qubit[], Qubit) => Unit is Adj), qs : Qubit[]) : Unit is Adj {        
    op(Most(qs), Tail(qs));
}

operation TestCompoundGate () : Unit {
    AssertOperationsEqualReferenced(6+1, QubitArrayWrapperOperation(Task3, _), QubitArrayWrapperOperation (original, _));
}

In [ ]:
TestCompoundGate();

In [19]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task3_DumpMachineWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task3(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task3_ResourceEstimationWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    Task3(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [20]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
# qsharp.config["dump.truncateSmallAmplitudes"]=True
Task3_DumpMachineWrapper.simulate()

|0000000⟩	0.12500000000000008 + 0𝑖
|0000001⟩	0 + 0𝑖
|0000010⟩	0.12500000000000008 + 0𝑖
|0000011⟩	0 + 0𝑖
|0000100⟩	0.12500000000000008 + 0𝑖
|0000101⟩	0 + 0𝑖
|0000110⟩	0 + 0𝑖
|0000111⟩	0.12500000000000008 + 0𝑖
|0001000⟩	0.12500000000000008 + 0𝑖
|0001001⟩	0 + 0𝑖
|0001010⟩	0.12500000000000008 + 0𝑖
|0001011⟩	0 + 0𝑖
|0001100⟩	0.12500000000000008 + 0𝑖
|0001101⟩	0 + 0𝑖
|0001110⟩	0 + 0𝑖
|0001111⟩	0.12500000000000008 + 0𝑖
|0010000⟩	0.12500000000000008 + 0𝑖
|0010001⟩	0 + 0𝑖
|0010010⟩	0.12500000000000008 + 0𝑖
|0010011⟩	0 + 0𝑖
|0010100⟩	0.12500000000000008 + 0𝑖
|0010101⟩	0 + 0𝑖
|0010110⟩	0 + 0𝑖
|0010111⟩	0.12500000000000008 + 0𝑖
|0011000⟩	0 + 0𝑖
|0011001⟩	0.12500000000000008 + 0𝑖
|0011010⟩	0 + 0𝑖
|0011011⟩	0.12500000000000008 + 0𝑖
|0011100⟩	0 + 0𝑖
|0011101⟩	0.12500000000000008 + 0𝑖
|0011110⟩	0.12500000000000008 + 0𝑖
|0011111⟩	0 + 0𝑖
|0100000⟩	0.12500000000000008 + 0𝑖
|0100001⟩	0 + 0𝑖
|0100010⟩	0.12500000000000008 + 0𝑖
|0100011⟩	0 + 0𝑖
|0100100⟩	0.12500000000000008 + 0𝑖
|0100101⟩	0 + 0𝑖
|0100110⟩	0 + 0𝑖
|0100111⟩	0.12500000000000008 + 0𝑖
|0101000⟩	0.12500000000000008 + 0𝑖
|0101001⟩	0 + 0𝑖
|0101010⟩	0.12500000000000008 + 0𝑖
|0101011⟩	0 + 0𝑖
|0101100⟩	0.12500000000000008 + 0𝑖
|0101101⟩	0 + 0𝑖
|0101110⟩	0 + 0𝑖
|0101111⟩	0.12500000000000008 + 0𝑖
|0110000⟩	0.12500000000000008 + 0𝑖
|0110001⟩	0 + 0𝑖
|0110010⟩	0.12500000000000008 + 0𝑖
|0110011⟩	0 + 0𝑖
|0110100⟩	0.12500000000000008 + 0𝑖
|0110101⟩	0 + 0𝑖
|0110110⟩	0 + 0𝑖
|0110111⟩	0.12500000000000008 + 0𝑖
|0111000⟩	0 + 0𝑖
|0111001⟩	0.12500000000000008 + 0𝑖
|0111010⟩	0 + 0𝑖
|0111011⟩	0.12500000000000008 + 0𝑖
|0111100⟩	0 + 0𝑖
|0111101⟩	0.12500000000000008 + 0𝑖
|0111110⟩	0.12500000000000008 + 0𝑖
|0111111⟩	0 + 0𝑖
|1000000⟩	0.12500000000000008 + 0𝑖
|1000001⟩	0 + 0𝑖
|1000010⟩	0.12500000000000008 + 0𝑖
|1000011⟩	0 + 0𝑖
|1000100⟩	0.12500000000000008 + 0𝑖
|1000101⟩	0 + 0𝑖
|1000110⟩	0 + 0𝑖
|1000111⟩	0.12500000000000008 + 0𝑖
|1001000⟩	0.12500000000000008 + 0𝑖
|1001001⟩	0 + 0𝑖
|1001010⟩	0.12500000000000008 + 0𝑖
|1001011⟩	0 + 0𝑖
|1001100⟩	0.12500000000000008 + 0𝑖
|1001101⟩	0 + 0𝑖
|1001110⟩	0 + 0𝑖
|1001111⟩	0.12500000000000008 + 0𝑖
|1010000⟩	0.12500000000000008 + 0𝑖
|1010001⟩	0 + 0𝑖
|1010010⟩	0.12500000000000008 + 0𝑖
|1010011⟩	0 + 0𝑖
|1010100⟩	0.12500000000000008 + 0𝑖
|1010101⟩	0 + 0𝑖
|1010110⟩	0 + 0𝑖
|1010111⟩	0.12500000000000008 + 0𝑖
|1011000⟩	0 + 0𝑖
|1011001⟩	0.12500000000000008 + 0𝑖
|1011010⟩	0 + 0𝑖
|1011011⟩	0.12500000000000008 + 0𝑖
|1011100⟩	0 + 0𝑖
|1011101⟩	0.12500000000000008 + 0𝑖
|1011110⟩	0.12500000000000008 + 0𝑖
|1011111⟩	0 + 0𝑖
|1100000⟩	0 + 0𝑖
|1100001⟩	0.12500000000000008 + 0𝑖
|1100010⟩	0 + 0𝑖
|1100011⟩	0.12500000000000008 + 0𝑖
|1100100⟩	0 + 0𝑖
|1100101⟩	0.12500000000000008 + 0𝑖
|1100110⟩	0.12500000000000008 + 0𝑖
|1100111⟩	0 + 0𝑖
|1101000⟩	0 + 0𝑖
|1101001⟩	0.12500000000000008 + 0𝑖
|1101010⟩	0 + 0𝑖
|1101011⟩	0.12500000000000008 + 0𝑖
|1101100⟩	0 + 0𝑖
|1101101⟩	0.12500000000000008 + 0𝑖
|1101110⟩	0.12500000000000008 + 0𝑖
|1101111⟩	0 + 0𝑖
|1110000⟩	0 + 0𝑖
|1110001⟩	0.12500000000000008 + 0𝑖
|1110010⟩	0 + 0𝑖
|1110011⟩	0.12500000000000008 + 0𝑖
|1110100⟩	0 + 0𝑖
|1110101⟩	0.12500000000000008 + 0𝑖
|1110110⟩	0.12500000000000008 + 0𝑖
|1110111⟩	0 + 0𝑖
|1111000⟩	0.12500000000000008 + 0𝑖
|1111001⟩	0 + 0𝑖
|1111010⟩	0.12500000000000008 + 0𝑖
|1111011⟩	0 + 0𝑖
|1111100⟩	0.12500000000000008 + 0𝑖
|1111101⟩	0 + 0𝑖
|1111110⟩	0 + 0𝑖
|1111111⟩	0.12500000000000008 + 0𝑖

()

## Step 3. Evaluate the code using resource estimation

In [21]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/6d28879c-7c46-4b13-98e8-7da92da7e95e/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iQuHackTest",
    location="eastus",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace iQuHackTest in location eastus.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 266039},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 362072},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 3},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 257411},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 4298},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 139},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current

In [22]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [23]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task3_ResourceEstimationWrapper, jobName="RE for the task 3")

Submitting Task3_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 3
   Job ID: e8026f9b-c0de-4595-8342-521fa85c0639
Waiting up to 30 seconds for Azure Quantum job to complete...
[7:24:43 PM] Current job status: Executing
[7:24:48 PM] Current job status: Succeeded


In [24]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,50766,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 3726 physical qubits to implement the algorithm logic, and 47040 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,39us 600ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (3us 600ns) multiplied by the 9 logical cycles to run the algorithm. If however the duration of a single T factory (here: 36us 400ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,23,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 7\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 23$ logical qubits."
Algorithmic depth,9,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 3 CCZ and 0 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,11,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 9. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,12,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 3 CCZ and 0 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,12,Number of T factories capable of producing the demanded 12 T states during the algorithm's runtime The total number of T factories 12 that are executed in parallel is computed as \(\left\lceil\dfrac{12\;\text{T states} \cdot 36us 400ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 39us 600ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,1,"Number of times all T factories are invoked In order to prepare the 12 T states, the 12 copies of the T factory are repeatedly invoked 1 times."
Physical algorithmic qubits,3726,Number of physical qubits for the algorithm after layout The 3726 are the product of the 23 logical qubits after layout and the 162 physical qubits that encode a single logical qubit.
Physical T factory qubits,47040,"Number of physical qubits for the T factories Each T factory requires 3920 physical qubits and we run 12 in parallel, therefore we need $47040 = 3920 \cdot 12$ qubits."
Required logical qubit error rate,1.98e-6,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 23 logical qubits and the total cycle count 11.


In [25]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [26]:
evaluate_results(result)

Logical algorithmic qubits = 23
Algorithmic depth = 9
Score = 207


207